In [1]:
import math
import random
import pygame
from pygame import mixer

# Initialize the pygame modules
pygame.init()

# Create the screen
screen = pygame.display.set_mode((800, 600))

# Background (loads background image)
background = pygame.image.load('background.png')

# Sound effects
mixer.music.load("background.mp3")
mixer.music.play(-1)  # Plays the music in an infinite loop

# Caption and Icon
pygame.display.set_caption("Space Invader")
icon = pygame.image.load('ufo.png')  # Game icon
pygame.display.set_icon(icon)

# Player
playerImg = pygame.image.load('spaceship.png')
playerX = 360  # Positioned initially at (360, 500)
playerY = 500
playerX_change = 0  # Determines horizontal movement (left or right)

# Enemy
enemyImg = []
enemyX = []
enemyY = []
enemyX_change = []
enemyY_change = []
num_of_enemies = 6
enemy_speed = 0.5  # Set a uniform speed for all enemies

def initialize_enemies():
    #Initialize or reset enemies to their default state.
    for i in range(num_of_enemies):
        enemyImg.append(pygame.image.load('monster.png'))
        x_position = random.randint(0, 736)
        y_position = random.randint(50, 150)
        enemyX.append(x_position)
        enemyY.append(y_position)
        enemyX_change.append(enemy_speed)  # Set uniform horizontal speed
        enemyY_change.append(30)  # Slightly slower descent

initialize_enemies()

# Bullet
bulletImg = pygame.image.load('bullet.png')
bulletX = 0
bulletY = 480
bulletX_change = 0
bulletY_change = 7
bullet_state = "ready"  # "ready" or "fire"

# Score
score_value = 0
font = pygame.font.Font('freesansbold.ttf', 32)
testX = 10
testY = 10

# Game Over
over_font = pygame.font.Font('freesansbold.ttf', 64)
restart_font = pygame.font.Font('freesansbold.ttf', 28)
game_over = False

def show_score(x, y):
    score = font.render("Score : " + str(score_value), True, (255, 255, 255))
    screen.blit(score, (x, y))

def game_over_text():
    over_text = over_font.render("GAME OVER", True, (255, 255, 255))
    screen.blit(over_text, (200, 250))
    restart_text = restart_font.render("Press R to Restart", True, (255, 255, 255))
    screen.blit(restart_text, (230, 320))

def player(x, y):
    screen.blit(playerImg, (x, y))

def enemy(x, y, i):
    screen.blit(enemyImg[i], (x, y))

def fire_bullet(x, y):
    global bullet_state
    bullet_state = "fire"
    screen.blit(bulletImg, (x + 16, y + 10))

def isCollision(enemyX, enemyY, bulletX, bulletY):
    distance = math.sqrt(math.pow(enemyX - bulletX, 2) + (math.pow(enemyY - bulletY, 2)))
    return distance < 27

def reset_game():
    global playerX, playerX_change, bulletY, bullet_state, score_value, game_over
    global enemyImg, enemyX, enemyY, enemyX_change, enemyY_change

    playerX = 360
    playerX_change = 0
    bulletY = 480
    bullet_state = "ready"
    score_value = 0
    game_over = False

    # Clear enemy lists and reinitialize enemies
    enemyImg.clear()
    enemyX.clear()
    enemyY.clear()
    enemyX_change.clear()
    enemyY_change.clear()
    initialize_enemies()

# Game Loop
running = True
while running:
    # RGB Background
    screen.fill((0, 0, 0))
    screen.blit(background, (0, 0))

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                playerX_change = -0.5
            if event.key == pygame.K_RIGHT:
                playerX_change = 0.5
            if event.key == pygame.K_SPACE:
                if bullet_state == "ready" and not game_over:
                    bulletSound = mixer.Sound("laser.mp3")
                    bulletSound.play()
                    bulletX = playerX
                    fire_bullet(bulletX, bulletY)
            if event.key == pygame.K_r and game_over:
                reset_game()

        if event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT:
                playerX_change = 0

    # Player Movement
    playerX += playerX_change
    if playerX <= 0:
        playerX = 0
    elif playerX >= 736:
        playerX = 736

    if not game_over:
        # Enemy Movement
        for i in range(num_of_enemies):
            # Check Game Over
            if enemyY[i] > 440:
                game_over = True
                for j in range(num_of_enemies):
                    enemyY[j] = 2000
                break

            enemyX[i] += enemyX_change[i]
            if enemyX[i] <= 0:
                enemyX_change[i] = enemy_speed  # Keep the uniform speed
                enemyY[i] += enemyY_change[i]
            elif enemyX[i] >= 736:
                enemyX_change[i] = -enemy_speed  # Keep the uniform speed
                enemyY[i] += enemyY_change[i]

            # Collision Detection
            collision = isCollision(enemyX[i], enemyY[i], bulletX, bulletY)
            if collision:
                explosionSound = mixer.Sound("explosion.mp3")
                explosionSound.play()
                bulletY = 480
                bullet_state = "ready"
                score_value += 1
                enemyX[i] = random.randint(0, 736)
                enemyY[i] = random.randint(50, 150)

            enemy(enemyX[i], enemyY[i], i)

        # Bullet Movement
        if bulletY <= 0:
            bulletY = 480
            bullet_state = "ready"

        if bullet_state == "fire":
            fire_bullet(bulletX, bulletY)
            bulletY -= bulletY_change

    # Draw Player
    player(playerX, playerY)

    # Show Score
    show_score(testX, testY)

    # Show Game Over Text
    if game_over:
        game_over_text()

    pygame.display.update()

pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html
